In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as io

import requests

In [2]:
df_raw = pd.read_csv("https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv&filename=time_series_covid19_confirmed_global.csv")
df_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,584,609,634,663,678,712,726,736,750,766
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2718,2811,2910,3007,3127,3256,3382,3517,3649,3848
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,717,717,723,723,731,738,738,743,743,743
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,24,24,25,25,25,25,26,27,27,27


In [3]:
df = df_raw[(df_raw["Lat"]!=0) & (df_raw["Long"]!=0)]
dates = list(df.columns)[4:]

In [12]:
scale = 2 * max(df[dates[-1]]) / (10**2)
fig = go.Figure(go.Scattergeo(
    lat = df["Lat"], lon = df["Long"],
    marker = dict(size = df[dates[-1]]/scale)
))
fig.show()

In [5]:
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,584,609,634,663,678,712,726,736,750,766
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2718,2811,2910,3007,3127,3256,3382,3517,3649,3848
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,717,717,723,723,731,738,738,743,743,743
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,24,24,25,25,25,25,26,27,27,27


In [38]:
def preprocess(df):
    ani_df = pd.DataFrame(columns=list(df.columns[:4])+["Date","Cumulative Cases"])
    dates = df.columns[4:]
    basic_df = df[df.columns[:4]].copy()
    names = []
    for i in range(df.shape[0]):
        name = (df.iloc[i,0] + ", ") if type(df.iloc[i,0])==str else ""
        name += df.iloc[i,1]
        names.append(name)
    basic_df["Name"] = names
    for d in dates:
        temp_df = basic_df.copy()
        temp_df["Date"] = d
        temp_df["Cumulative Cases"] = pd.to_numeric(df[d],downcast="integer")
        ani_df = pd.concat([ani_df,temp_df.copy()],ignore_index=False)
    return ani_df

In [39]:
ani_df = preprocess(df)
ani_df.tail()

,Province/State,Country/Region,Lat,Long,Date,Cumulative Cases,Name
259,Saint Pierre and Miquelon,France,46.885200,-56.315900,4/29/20,1,"Saint Pierre and Miquelon, France"
260,NaN,South Sudan,6.877000,31.307000,4/29/20,34,South Sudan
261,NaN,Western Sahara,24.215500,-12.885800,4/29/20,6,Western Sahara
262,NaN,Sao Tome and Principe,0.186360,6.613081,4/29/20,8,Sao Tome and Principe
263,NaN,Yemen,15.552727,48.516388,4/29/20,6,Yemen


In [41]:
MAXSIZE = 100
scale = 2 * max(ani_df["Cumulative Cases"]) / (MAXSIZE**2)

fig = px.scatter_geo(
    data_frame = ani_df
    , lat = "Lat"
    , lon = "Long"
    , size = list(ani_df["Cumulative Cases"] / scale)
    , size_max = MAXSIZE
    , animation_frame="Date"
    , hover_data = ani_df[["Name","Cumulative Cases"]]
)

fig.update_layout(
    geo = go.layout.Geo(
        scope = 'world',
        landcolor = "rgb(229, 229, 229)",
        showcountries = True,
        showsubunits = True,
        bgcolor = 'rgba(0, 255, 0, 0.0)',
        subunitcolor = 'rgba(255, 255, 255, 0.0)'
    ),
)

fig.show()
fig.write_html("Test.html")